In [1]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file

In [9]:
data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2'  # external date only drive
data_dir = os.getenv('FINANCE_DATA') #internal folder with finance data 
symbols = [s for s in os.listdir(data_dir) if s.endswith('.L')]

In [17]:
labels = os.path.join(data_dir,'Labels')

idxSymbol = 1 #0,1,2, in our case
print("you picked symbol: ",symbols[idxSymbol]) 

featuresDates = os.path.join(data_dir,symbols[idxSymbol],'MODEL_BASED') # path all the list of dates we have features for
print('-------------')
print(featuresDates)
print('-------------')
print('Features Dates:')
print('-------------')
featuresDateList = os.listdir(os.path.join(data_dir,symbols[idxSymbol],'MODEL_BASED')) #list of all dates with features
print(featuresDateList)
print('-------------')

idxDate= 9 #as an example, enumerate the above list when parsing thru

featuresDateItems = os.path.join(os.path.join(data_dir,symbols[idxSymbol],'MODEL_BASED'), featuresDateList[idxDate])
#for each of the featuresDateItems, I have a set of features that needs to get matched with a set of labels and used to train an MKLSVM
print(featuresDateItems)
print('---------------')
listFeaturesPickle = os.listdir(featuresDateItems) #all the features!


you picked symbol:  PRU.L
-------------
/home/ak/FinData/PRU.L/MODEL_BASED
-------------
Features Dates:
-------------
['20180417', '20170724', '20180418', '20170713', '20170120', '20170811', '20170808', '20180406', '20180226', '20180202', '20180214', '20180404', '20180212', '20170818', '20170809', '20180201', '20170823', '20180213', '20180419', '20170130', '20170719', '20170804', '20170705', '20170117', '20170726', '20170126', '20170718', '20170810', '20170707', '20170301', '20170711', '20180227', '20170703', '20170131', '20170825', '20180221', '20180223', '20170714', '20170728', '20180206', '20170803', '20170123', '20180405', '20180413', '20170801', '20170807', '20170727', '20170821', '20170815', '20170124', '20180205', '20180403', '20180410', '20180222', '20170127', '20180228', '20170116', '20180409', '20170817', '20170829', '20170731', '20180208', '20180416', '20170119', '20170704', '20170725', '20180412', '20170721', '20180207', '20170706', '20170822', '20180216', '20170824', '201

In [16]:
print(featuresDateList[idxDate])
os.listdir(featuresDateItems)

20180202


['PRU.L_3_states_features_date:_20180403_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180404_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180208_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180412_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180215_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180226_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180227_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180213_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180406_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180409_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180416_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180228_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180411_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180417_now:_20181229_.pickle',
 'PRU.L_3_states_features_date:_20180212_now:_20181229_.pickle',
 'PRU.L_3_states_features

In [ ]:
idxFeatures = 0 #placeholder for enumerator
featuresTuple = pickle.load(open(os.path.join(featuresDateItems,listFeaturesPickle[idxFeatures]), "rb"), encoding='latin1')

In [ ]:
featuresArray = np.asarray(pd.concat([featuresTuple[0], featuresTuple[1],\
                                                 featuresTuple[2], featuresTuple[3]], axis=1, sort=False).fillna(0))

In [ ]:
labelsDate = os.path.splitext(listFeaturesPickle[idxDate])[0].split("_")[5]
#take the featuresTuple file, pick the first element and split it using "_", then pick the data the features were created

symbolLabelDir = os.path.join(labels,symbols[idxSymbol],'NON_DIRECTIONAL') #path that has all the labels for the particular symbol, i.e all the dates
#each day is a set of labels

In [ ]:
labelsFile = os.path.join(symbolLabelDir,labelsDate+".csv") # formulate the path for (as an example) the specific date we dealt with

In [ ]:
labelsDf= pd.read_csv(labelsFile).fillna(0) #load the dataframe and replace the NAs with zero (this is small cheat)
labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] #just retain the column of labels please

In [ ]:
#formulate the problem
Y_all = labels
X_all = featuresArray

In [ ]:
X_all.shape[0] == Y_all.shape[0] #check for dimensions, if False, there is a problem somewhere, prob picked wrong date

In [ ]:
labelsDf.shape